In [1]:
import rioxarray

def reproject_and_align_pop(factor, base_dataset):
    #Pop dataset is in 30-Arc-seconds resolution, change the factor accordingly

    pop = rioxarray.open_rasterio('../files/LandScanGlobal/landscan-global-2024.tif')
    pop = pop.rename({'y': 'latitude', 'x': 'longitude'})
    pop_us = pop.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 

    #Replace fill value (a negative int) with nan values
    fill_value = pop_us.attrs['_FillValue']
    pop_us_clean = pop_us.where(pop_us != fill_value)

    #sum aggregated for the population
    pop_agg = pop_us_clean.coarsen(
        latitude=factor, 
        longitude=factor, 
        boundary='trim'
    ).sum()

    #then we allign the grids
    pop_regridded = pop_agg.interp(
        latitude=base_dataset.latitude,
        longitude=base_dataset.longitude,
        method='nearest'
    )

    reprojected_deg = factor * 1/120 #30 arc seconds  = 1/200 deg

    pop_regridded_clean = pop_regridded.squeeze() #remove the band of dimension 1

    pop_regridded_clean.name = 'Population'
    pop_regridded_clean.attrs["grid"] = f"{reprojected_deg} degree"
    pop_regridded_clean.attrs["created"] = "2026-01-29"

    pop_regridded_clean.to_netcdf(f"../files/population_regridded_{str(reprojected_deg).replace('.', '')}deg.nc")

    return pop_regridded_clean

Population to ERA5-Land Daily Average reprojection (0.1°)

In [ ]:
import xarray as xr

ds = xr.open_dataset(
    "https://data.earthdatahub.destine.eu/era5/era5-land-daily-utc-v1.zarr",
    storage_options={"client_kwargs":{"trust_env":True}},
    chunks={},
    engine="zarr",
)

#Reassign this dataset longitude to be -180, 180
if ds.longitude.max() > 180:
    ds = ds.assign_coords(longitude=((ds.longitude + 180) % 360) - 180)
    ds = ds.sortby("longitude")
ds = ds.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 

pop = reproject_and_align_pop(12, ds)

Population to ECMWF OPEN DATA IFS forecasts (0.25°)

In [ ]:
from ecmwf.opendata import Client
import xarray as xr

client = Client()

result = client.retrieve(
    date=0, 
    time=0,
    type="fc",
    stream='oper',
    step=0,
    param='2t',
    target="reproject.grib2",
)

ds = xr.open_dataset('reproject.grib2')
ds = ds.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 

pop = reproject_and_align_pop(30, ds)